# LUIS sdk

Notebook para exploración del servicio de LUIS para enriquecer la tabla storage del Bot.

In [ ]:
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
import requests
import json
from types import SimpleNamespace

In [ ]:
#----------- Credenciales
authoringKey = 'ad5103875f974213af5db4ca3d997460'
authoringEndpoint = 'https://cog-luis-fna-parce-authoring.cognitiveservices.azure.com/'

app_id = "7e20d5a3-2949-449d-b550-0f2e66ddaa40"
predictionKey = 'bc07e0dc5d25489091aad03387caf06a'
predictionEndpoint = 'https://EastUS2.api.cognitive.microsoft.com/'

#----------- Client
client = LUISAuthoringClient(authoringEndpoint, CognitiveServicesCredentials(authoringKey))

#----------- Client runtime
runtimeCredentials = CognitiveServicesCredentials(predictionKey)
clientRuntime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtimeCredentials)

In [ ]:
def LUIS(x):
    x = str(x)

    #----------- Configuración del REST CALL
    headers = {}
    params ={'query': x, 'subscription-key': predictionKey}

    # Make the REST call.
    response = requests.get(f'{predictionEndpoint}luis/prediction/v3.0/apps/{app_id}/slots/production/predict'
                            , headers=headers
                            , params=params)
    
    from types import SimpleNamespace    
    a = response.json()
    m = SimpleNamespace(**a)

    b = m.prediction
    n = SimpleNamespace(**b)

    c = n.intents
    o = SimpleNamespace(**c)

    k = list(c.values())
    p = k[0]
    q = SimpleNamespace(**p)
    
    #, q.score
    return str(n.topIntent)+'-'+str(q.score)

In [ ]:
print(LUIS('Hola, como encuentro un formulario para retirar mis cesantias'))

In [ ]:
entradas = [(1,'Hola, como encuentro un formulario para retirar mis cesantias')
               ,(2,'Hola, como encuentro un formulario para declaracion de renta')]
entradas

In [ ]:
#-- Crear DF
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, FloatType

# Create a schema for the dataframe
schema = StructType([
    StructField('Id', IntegerType(), True),
    StructField('Texto', StringType(), True)
])

# Convert list to RDD
rdd = spark.sparkContext.parallelize(entradas)

# Create data frame
df01 = spark.createDataFrame(rdd,schema)
df01.show()

In [ ]:
# Convert a Python function to PySpark UDF

from pyspark.sql.functions import udf, col

""" Converting function to UDF """
LUIS_UDF = udf(lambda z: LUIS(z),StringType())

In [ ]:
# Aplicar la funcion UDF de LUIS a la columna de usuario
df01 = df01.withColumn("LUIS", LUIS_UDF(col("Texto")))

In [ ]:
import pyspark

# Split de la columna LUIS
split_cols = pyspark.sql.functions.split(df01['LUIS'], '-')
  
# Aplicar split con '-'
df = df01.withColumn('appLUIS', split_cols.getItem(0)).withColumn('LUIS_score', split_cols.getItem(1))

# Eliminar col LUIS
df = df.drop('LUIS')

# Ajustar tipo de col en score
df = df.withColumn("LUIS_score", df["LUIS_score"].cast(FloatType()))

In [ ]:
df.printSchema()
df.show()